## 1.Importing Libraries

In [1]:
import sys
import os

import gc
import warnings

from subprocess import check_output


from datetime import datetime
from glob import glob

import matplotlib.pyplot as plt

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as se

import scipy.stats as stats

from pandas_profiling import ProfileReport


In [2]:
#se.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})
#Set pandas view options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
warnings.filterwarnings('ignore')
#%matplotlib inline

## 2.Load Data

In [3]:
raw_data_path = '/Users/Artem_Boltaev/Documents/EPAM Projects/6. RecSys_course/source_code/recsys_course_epam/data/raw/instacart-market-basket-analysis'
print(check_output(["ls", raw_data_path]).decode("utf8"))

aisles.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
products.csv
sample_submission.csv



In [4]:
order_products_train_df = pd.read_csv(raw_data_path + "/order_products__train.csv")
order_products_prior_df = pd.read_csv(raw_data_path + "/order_products__prior.csv")
orders_df = pd.read_csv(raw_data_path + "/orders.csv")
products_df = pd.read_csv(raw_data_path + "/products.csv")
aisles_df = pd.read_csv(raw_data_path + "/aisles.csv")
departments_df = pd.read_csv(raw_data_path + "/departments.csv")
sample_submission_df = pd.read_csv(raw_data_path + "/sample_submission.csv")

1 . aisles.csv - aisles in which the products belong

2 . departments.csv - department in which the products belong

3 . order_products__prior.csv - These files specify which products were purchased in each order. order_products__prior.csv contains previous order contents for all customers. 'reordered' indicates that the customer has a previous order that contains the product. Note that some orders will have no reordered items.

4 . order_products__train.csv - These files specify which products were purchased in each order. order_products__prior.csv contains previous order contents for all customers. 'reordered' indicates that the customer has a previous order that contains the product. Note that some orders will have no reordered items.

5 . orders.csv - This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders. 'order_dow' is the day of week.

6 . products.csv - list of all products

7 . sample_submission.csv - a sample submission file in the correct format

In [6]:
#merge all dataframes into one and make ProfileReport
df_prior = pd.merge(order_products_prior_df, products_df, on='product_id', how='left')
df_prior = pd.merge(df_prior, aisles_df, on='aisle_id', how='left')
df_prior = pd.merge(df_prior, departments_df, on='department_id', how='left')
df_prior = pd.merge(df_prior, orders_df, on='order_id', how='left')
df_prior.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs,202279,prior,3,5,9,8.0
1,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables,produce,202279,prior,3,5,9,8.0
2,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry,202279,prior,3,5,9,8.0
3,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry,202279,prior,3,5,9,8.0
4,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry,202279,prior,3,5,9,8.0


In [7]:
df_train = pd.merge(order_products_train_df, products_df, on='product_id', how='left')
df_train = pd.merge(df_train, aisles_df, on='aisle_id', how='left')
df_train = pd.merge(df_train, departments_df, on='department_id', how='left')
df_train = pd.merge(df_train, orders_df, on='order_id', how='left')
df_train.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,112108,train,4,4,10,9.0
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs,112108,train,4,4,10,9.0
2,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
3,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,112108,train,4,4,10,9.0


In [8]:
df = pd.concat([df_prior, df_train], axis=0)
df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs,202279,prior,3,5,9,8.0
1,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables,produce,202279,prior,3,5,9,8.0
2,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry,202279,prior,3,5,9,8.0
3,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry,202279,prior,3,5,9,8.0
4,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry,202279,prior,3,5,9,8.0


In [11]:
df_mod = df.copy()
df_mod['product_count_by_user'] = df_mod.groupby(['user_id', 'product_id'])['product_id'].transform('count')
df_mod = df_mod.drop_duplicates(subset=['user_id', 'product_id'])
df_mod.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_count_by_user
0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs,202279,prior,3,5,9,8.0,6
1,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables,produce,202279,prior,3,5,9,8.0,5
2,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry,202279,prior,3,5,9,8.0,1
3,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry,202279,prior,3,5,9,8.0,5
4,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry,202279,prior,3,5,9,8.0,3


In [13]:
#Rating matrix
rating_matrix = df_mod.pivot(index="user_id", columns="product_id", values="product_count_by_user").fillna(0)